# Example using a HLS IP with AXI Master from PYNQ

## Import and download the HLS example Overlay

In [1]:
from pynq import Overlay
ol = Overlay("hls_m_axi.bit")

## Examine the design

In [2]:
ol.ip_dict

{'axi_intc_0': {'fullpath': 'axi_intc_0',
  'type': 'xilinx.com:ip:axi_intc:4.1',
  'bdtype': None,
  'state': None,
  'addr_range': 65536,
  'phys_addr': 1092616192,
  'mem_id': 's_axi',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {'intr': {'controller': 'axi_intc_0',
    'index': 0,
    'fullpath': 'axi_intc_0/intr'}},
  'parameters': {'C_FAMILY': 'zynq',
   'C_INSTANCE': 'hls_m_axi_axi_intc_0_0',
   'C_S_AXI_ADDR_WIDTH': '9',
   'C_S_AXI_DATA_WIDTH': '32',
   'C_NUM_INTR_INPUTS': '1',
   'C_NUM_SW_INTR': '0',
   'C_KIND_OF_INTR': '0xfffffffe',
   'C_KIND_OF_EDGE': '0xFFFFFFFF',
   'C_KIND_OF_LVL': '0xFFFFFFFF',
   'C_ASYNC_INTR': '0xFFFFFFFF',
   'C_NUM_SYNC_FF': '2',
   'C_ADDR_WIDTH': '32',
   'C_IVAR_RESET_VALUE': '0x0000000000000010',
   'C_ENABLE_ASYNC': '0',
   'C_HAS_IPR': '1',
   'C_HAS_SIE': '1',
   'C_HAS_CIE': '1',
   'C_HAS_IVR': '1',
   'C_HAS_ILR': '0',
   'C_IRQ_IS_LEVEL': '1',
   'C_IRQ_ACTIVE': '0x1',
   'C_DISABLE_SYNCHRONIZERS': '0',
   'C_MB_CLK_NOT_CO

## Create an alias for the IP

In [3]:
my_ip = ol.example_0

## Check the register map for the HLS IP

You should see the registers that we need to write values to:

* CTRL - the control register
* a_1 - the lower 32-bits of the start address for the memory range the HLS IP will access
* length_r - the range of the memory buffer
* value - the value that will be added to each memory location

In [4]:
my_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, RESERVED_3=0, RESERVED_4=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  a_1 = Register(a=write-only),
  a_2 = Register(a=write-only),
  length_r = Register(length_r=write-only),
  value_r = Register(value_r=write-only)
}

## Allocate some memory

The start address for this memory will be passed to the HLS IP. The HLS IP will read data from this memory and write results back to the same locations. 

We can initialise the memory 

In [16]:
from pynq import allocate
import numpy as np 
py_buffer = allocate(shape=(100,), dtype=np.int32)

In [17]:
print("py_buffer physical address {}".format(hex(py_buffer.physical_address)))

py_buffer physical address 0x16849000


## Configure the IP

Write the values of the memory address, the length, and the value to be added to each location.
You must make sure that length doesn't exceed the lendth of the memory that we allocated earlier AND the maximum value that can be set in the HLS IP. The buffe

In [12]:
my_ip.register_map.a_1 = py_buffer.physical_address
my_ip.register_map.length_r = 100
my_ip.register_map.value_r = 7 

## Start the IP

In [13]:
my_ip.register_map.CTRL.AP_START=1

## Check the IP completed

If we start the IP once, when the IP completes an iteration DONE will go high **once**. While the IP is working, AP_IDLE will be 0. In this example, the time for an iteration of the HLS IP is very low so you will probably not see AP_IDLE = 0. 

Done is cleared when it is read. There is a limitation with the register_map. Multiple reads to the same register may be made to collect the information required for the register_map. This may mean DONE is cleared while the values of some of the other bits and registers are being read and consequentally will not show as high in the register map.


In [14]:
my_ip.register_map.CTRL.AP_DONE

0

If you have a long running HLS IP, you can poll on the DONE bit before continuing with your application. 

## Check the contents of the memory

In [15]:
py_buffer

PynqBuffer([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0

In [11]:
my_ip.register_map.CTRL.AUTO_RESTART=1